# Blotto

## Rules:
There are 10 castles, 2 players, each with 100 soldiers.
Simultaneously, each player allocates some number of soldiers to each castle.
For each castle, the player who allocated more soldiers to that castle "wins" that castle.
Castle `i` is worth `i` points. Ties evenly distribute points.
The goal may be to maximize probability of winning, or maximize expected number of points won.

In [1]:
import numpy as np

In [16]:
# in numpy batched operations, each row is a game.
# So a shape of an array may be (num_games, 10)
# Ideally, shape (10,) should be fine.
castle_points = np.arange(1, 11)


def get_points(a, b):
    # returns player a score.
    # Since sum of scores is 1 + ... + 10 = 55, easy to get b score
    wins = np.where(a > b, 1, 0) + np.where(a == b, 1 / 2, 0)
    return np.sum(wins * castle_points, axis=-1, keepdims=True)

## Simulation ideas:
The ultimate goal is to have the highest score or number of wins (to be determined) when matched against the pool of strategies submitted by other interns. This is not knowable. It is a good idea to brainstorm what strategies other interns may use. There may also be some database of strategies to compete against.

Ultimately, any Nash equilibrium for this game must be a mixed strategy. For a quick proof, consider a pure strategy. A strategy that beats this can be created by copying this strategy, increasing the soldiers allocated to castle 10, and decreasing the soldiers allocated to some other castle. The only way this process fails is if all soldiers are allocated to castle 10, and this is easy to beat.

But I/we cannot submit mixed strategies. 

A good question is, given a collection of pure strategies, come up with a pure strategy that maximizes the expected value of points or expected win rate against this collection. On the face, this seems hard: it appears to be a combinatorial optimization problem. Even integer programming is np hard, and our objective is nonlinear (although it is monotonic). We may be able to get somewhere by relaxing integrality and replacing the step function for the score with a sigmoid-esque function, allowing for gradient-based optimization.

Let's start with a genetic algorithm. We can make each organism a pure strategy or a mixed strategy. Let's start with pure.

In [146]:
def rank_population(population):  # population shape (pop_size, 10)
    """Takes population, returns array of shape (pop_size, 1) whose
    i-th entry is the index of the i-th best organism"""
    a = np.expand_dims(population, 1)  # shape (pop_size, 1, 10)
    b = np.expand_dims(population, 0)  # shape (1, pop_size, 10)
    points_by_game = get_points(a, b)
    total_points = np.sum(points_by_game, axis=1)  # shape (pop_size, 1)
    return np.argsort(total_points, axis=0)[::-1]  # sort descending by score

In [109]:
def mutate(population, gen, num_mutations=1):
    num_soldiers = np.sum(population, axis=-1)
    population = population.copy()
    # randomly move 1 soldier
    # choice to make: random by tower or random by soldier? I'll do by tower
    for _ in range(num_mutations):
        remove = gen.integers(population.shape[1], size=population.shape[0])
        add = gen.integers(population.shape[1], size=population.shape[0])
        idx = np.arange(population.shape[0])
        mutate_filter = population[idx, remove] > 0 # can't remove if 0 soldiers
        population[idx, remove] = np.where(
            mutate_filter,
            population[idx, remove] - 1,
            population[idx, remove]
        )
        population[idx, add] = np.where(
            mutate_filter, population[idx, add] + 1, population[idx, add]
        )
    assert np.all(np.sum(population, axis=-1) == num_soldiers) # assert soldiers preserved
    return population

In [118]:
def initialize(num_organisms, gen, num_castles=10, num_soldiers=100):
    population = np.zeros((num_organisms, num_castles), dtype=np.int8)
    for _ in range(num_soldiers):
        idx = np.arange(num_organisms)
        add = gen.integers(num_castles, size=num_organisms)
        population[idx, add] += 1
    return population

In [149]:
def genetic_step(population, gen):
    keep_ratio = 2 # keep 1/keep_ratio
    num_organisms = population.shape[0]
    rank = rank_population(population).reshape(-1) # shape (num_organisms)
    top_organisms = rank[:num_organisms//keep_ratio]
    top_organisms = population[top_organisms]
    new_population = np.tile(top_organisms, (keep_ratio, 1))
    return mutate(new_population, gen)

In [152]:
gen = np.random.default_rng(seed=1)
p = initialize(1000, gen, num_castles = 10, num_soldiers = 100)
for _ in range(100):
    p = genetic_step(p, gen)

In [153]:
p[:10, :]

array([[ 2,  2,  2, 10,  2, 16, 15, 13, 38,  0],
       [ 2,  1,  3, 12,  1, 15, 15, 12, 38,  1],
       [ 2,  1,  2, 10,  2, 17, 14, 13, 39,  0],
       [ 3,  1,  5,  1,  4, 16, 22,  5, 37,  6],
       [ 1,  2,  1, 11,  0, 18, 15, 12, 38,  2],
       [ 2,  0,  6,  1,  5, 16, 22,  6, 37,  5],
       [ 2,  2,  3, 10,  0, 16, 17, 12, 37,  1],
       [ 1,  2,  1, 10,  1, 17, 16, 12, 38,  2],
       [ 0,  3,  2,  8,  1, 17, 16, 12, 39,  2],
       [ 0,  2,  2,  9,  1, 17, 16, 12, 39,  2]], dtype=int8)

In [156]:
gen = np.random.default_rng(seed=2)
p = initialize(1000, gen, num_castles = 10, num_soldiers = 100)
for _ in range(100):
    p = genetic_step(p, gen)
p[:10, :]

array([[ 2,  0,  7,  9, 14,  3, 15, 17, 17, 16],
       [ 2,  0,  8,  9, 13,  3, 15, 18, 16, 16],
       [ 1,  0,  6, 10, 15,  1, 19, 18, 16, 14],
       [ 2,  0,  7,  9, 14,  2, 16, 17, 17, 16],
       [ 0,  1,  6, 10, 15,  1, 19, 18, 16, 14],
       [ 1,  2,  6, 10, 15,  2, 20, 16, 16, 12],
       [ 0,  0,  7, 13, 13,  2, 18, 16, 18, 13],
       [ 1,  1,  6, 10, 14,  2, 19, 18, 15, 14],
       [ 1,  4,  3,  4,  7, 16, 10,  2, 19, 34],
       [ 1,  0,  6, 10, 15,  3, 18, 18, 16, 13]], dtype=int8)

In [157]:
gen = np.random.default_rng(seed=3)
p = initialize(1000, gen, num_castles = 10, num_soldiers = 100)
for _ in range(100):
    p = genetic_step(p, gen)
p[:10, :]

array([[ 1,  3,  5, 10, 11, 13, 18, 30,  1,  8],
       [ 0,  0,  9,  2,  2, 19, 10, 19, 18, 21],
       [ 0,  0,  9,  1,  2, 19, 12, 17, 18, 22],
       [ 0,  0,  9,  2,  1, 19, 12, 18, 19, 20],
       [ 0,  3,  8,  2,  1, 20,  8, 20, 17, 21],
       [ 1,  4,  6, 14,  6,  9, 20, 29,  2,  9],
       [ 1,  0,  8,  3,  2, 20, 12, 17, 18, 19],
       [ 0,  1,  8,  1,  0, 17, 13, 22, 18, 20],
       [ 0,  1,  8,  1,  2, 19, 10, 19, 18, 22],
       [ 1,  0,  8,  1,  5, 19, 12, 17, 18, 19]], dtype=int8)